In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor

## **Разделим данные на тренировочные и тестовые и сделаем конвеер для первичной обработки**

In [2]:
df_regr1 = pd.read_excel("data.xlsx")
train_regr1, test_regr1 = train_test_split(df_regr1, test_size=0.3, random_state=42)

#### **Класс для рассчета Si**

In [3]:
class Calc_Si(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X["SI"] = X["CC50, mM"] / X["IC50, mM"]
        return X

#### **Класс для удаления выбросов**

In [4]:
class Outliers_data(BaseEstimator, TransformerMixin):

    def __init__(self, target_cols, threshold=1000):
        self.columns =  target_cols #["IC50, mM", "SI"]
        self.threshold = threshold

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        self.filters_ = (X[self.columns] <= self.threshold).all(axis=1)
        return X[self.filters_].copy()
    

#### **Класс для логорифмирования целевой переменной**

In [5]:
class LogTargetTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, target_col, replace_zero=1e-10):
        self.target_col = target_col
        self.replace_zero = replace_zero
    
    def fit(self, X, y=None):
        return self  
    
    def transform(self, X):
        X_transformed = X.copy()
        # Заменяем нули и логарифмируем
        X_transformed[self.target_col] = np.log(X[self.target_col].replace(0, self.replace_zero))
        return X_transformed

#### **Класс для удаления нулевых признаков**

In [6]:
class DropZeroData(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_to_drop_ = None

    def fit(self, X, y=None):
        self.columns_to_drop_ = X.columns[X.sum() == 0].tolist()
        return self
    
    def transform(self, X):
        return X.drop(columns=self.columns_to_drop_)

#### **Класс для создания новых признаков**

In [7]:
class FeatureAggregator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):      
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        
        # Групповые суммы
        X_transformed['Hydrophilic_VSA'] = X[['SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3']].sum(axis=1)
        X_transformed['Moderate_VSA'] = X[['SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6']].sum(axis=1)
        X_transformed['Lipophilic_VSA'] = X[['SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA10']].sum(axis=1)
        X_transformed['frointier_VSA'] = X[['SlogP_VSA11', 'SlogP_VSA12']].sum(axis=1)
        
        # Разности
        X_transformed["DiffPartialCharge"] = X["MaxPartialCharge"] - X["MinPartialCharge"]
        X_transformed["DiffEStateIndex"] = X["MaxEStateIndex"] - X["MinEStateIndex"]
        
        return X_transformed

#### **Данные по матрице корреляции**

In [8]:
data_to_drop = ['fr_COO',
 'fr_COO2',
 'fr_Nhpyrrole',
 'fr_COO2',
 'fr_phenol_noOrthoHbond',
 'fr_phenol',
 'fr_phenol_noOrthoHbond',
 'fr_C_O_noCOO',
 'fr_Al_OH_noTert',
 'fr_nitro_arom_nonortho',
 'fr_ketone_Topliss',
 'fr_halogen',
 'fr_NH0','BCUT2D_MRHI', 'BCUT2D_LOGPLOW', 'BCUT2D_LOGPHI']

### **Класс для отбора признаков**

In [9]:
class MutualInfoFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, target_col='CC50, mM', threshold=0.1, random_state=42):
        self.target_col =target_col
        self.threshold = threshold
        self.random_state = random_state
        self.selected_features_ = None
        self.feature_importances_ = None
    
    def fit(self, X):
        y = X[self.target_col]
        X_mut = X.drop(columns = self.target_col) 
        # Вычисляем mutual information
        importances = mutual_info_regression(X_mut, y, random_state=self.random_state)
        
        # Сохраняем важность признаков
        self.feature_importances_ = pd.Series(importances, index=X_mut.columns)
        
        # Выбираем признаки, превышающие порог
        self.selected_features_ = self.feature_importances_[
            self.feature_importances_ >= self.threshold
        ].index.tolist()
        self.selected_features_.append(self.target_col)
        return self
    
    def transform(self, X):
        # Возвращаем только выбранные признаки

        return X[self.selected_features_]

### **Конвеер предобработки обработки данных**

In [ ]:
preprocessing_pipeline = Pipeline([
    ('drop Unnamed 0', FunctionTransformer(lambda df: df.drop(columns=["Unnamed: 0"]))), # удаляем Unnamed 
    ('drop_na', FunctionTransformer(lambda df: df.dropna())), #Удаляем строки с пропусками  
    ('drop_duplicates', FunctionTransformer(lambda df: df.drop_duplicates())), # Удаляем строки дубликаты
    ('calculate_si', Calc_Si()), # Класс для расчета Si
    ('drop CC50', FunctionTransformer(lambda df: df.drop(columns=['CC50, mM']))),
    ('filter_data', Outliers_data(target_cols=["SI"])),  # Класс для удаления выбросов
    ('log_target', LogTargetTransformer(target_col="SI")), # Логарифмируем целевую
    ('drop Zero', DropZeroData()),
    ("create new feature", FeatureAggregator()),
    ('drop corr data', FunctionTransformer(lambda df: df.drop(columns=data_to_drop))),
    ('select feature', MutualInfoFeatureSelector(target_col="SI", threshold=0.15))
])

In [78]:
encode_train = preprocessing_pipeline.fit_transform(train_regr1)
encode_test = preprocessing_pipeline.transform(test_regr1)

 # **Выбор модели и подбор гиперпараметров**

In [79]:
# Выделим X и Y
y_train = encode_train["SI"]
x_train = encode_train.drop(columns=["SI"])

y_test = encode_test["SI"]
x_test = encode_test.drop(columns=["SI"])

### **Стандартизируем отобранные данные**

In [80]:
scaler = StandardScaler()

X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)

### **Линейная регрессия**

In [81]:
# Создадим модель
lig_reg = LinearRegression()

# Кросс-валидация на тренировочных данных
cv_scores = cross_val_score(lig_reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("Точность на каждом из фолдов:\n", -cv_scores)
print("Средняя MSE на кросс-валидации:", -np.mean(cv_scores))
print("Среднее RMSE на кросс-валидации:", np.sqrt(-np.mean(cv_scores)))

# Обучаем модель 
lig_reg.fit(X_train, y_train)

# Предсказание на тестовых данных
Y_pred = lig_reg.predict(X_test)

# Оценка качества на тестовых данных
print("\nМетрики на тестовой выборке:")
print("----------------------------------------")

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, Y_pred)
print(f"MAE (Средняя абсолютная ошибка): {mae:.4f}")

# MSE (Mean Squared Error)
mse = mean_squared_error(y_test, Y_pred)
print(f"MSE (Средняя квадратичная ошибка): {mse:.4f}")

# RMSE (Root Mean Squared Error)
rmse = np.sqrt(mse)
print(f"RMSE (Корень из MSE): {rmse:.4f}")

# R² (Коэффициент детерминации)
r2 = r2_score(y_test, Y_pred)
print(f"R² (Коэффициент детерминации): {r2:.4f}")


Точность на каждом из фолдов:
 [1.75191117 2.36631849 2.2987993  2.0603671  2.21558087]
Средняя MSE на кросс-валидации: 2.1385953863768075
Среднее RMSE на кросс-валидации: 1.4623937179763893

Метрики на тестовой выборке:
----------------------------------------
MAE (Средняя абсолютная ошибка): 1.1162
MSE (Средняя квадратичная ошибка): 2.6981
RMSE (Корень из MSE): 1.6426
R² (Коэффициент детерминации): 0.1464


In [87]:
y_test.mean()

1.7403486981895804

#### **Оценка метрик**

**MAE** (Средняя абсолютная ошибка): 1.1162- модель в среднем ошибается на ±1.11 мМ  значение ошибки велико так как среднее значение  y_test = 1.74

**MSE** (Средняя квадратичная ошибка): 2.69 - так же указывает на то что в модели есть выбросы

**RMSE** (Корень из MSE): 1.64 - так же указывает на выбросы в модели

**R²** (Коэффициент детерминации): 0.1464 - модель обьясняет 15%  обьясненной дисперсии, это плохой результат

### **RandomForest**

In [82]:
# Выбираем гиперпараметры которые будем определять
parametrs = {
    'n_estimators': Integer(100, 1000),  # Число деревьев в "лесу"
    'max_depth': Integer(3, 20),       # Максимальная глубина дерева
    'min_samples_split': Integer(2, 10), # Минимальное количество объектов, необходимое для разделения внутреннего узла
    'min_samples_leaf': Integer(1, 10), # Минимальное число объектов в листе
    'max_features': Real(0.1, 1.0)     # число признаков, по которым ищется разбиение
}

# Модель
RF_model = RandomForestRegressor(random_state=42)

# Байесовская оптимизация
bayes_search = BayesSearchCV(
    RF_model,
    parametrs,
    n_iter=32,  # Количество итераций
    cv=5,       # Количество фолдов для кросс-валидации
    scoring='neg_mean_squared_error',
    random_state=42
)

# Поиск лучших параметров
bayes_search.fit(X_train, y_train)

print("\nЛучшие гиперпараметры:")
for key, val in bayes_search.best_params_.items():
    print(f'{key}: {val}')
print(f"Лучший MSE на кросс-валидации: {-bayes_search.best_score_:.4f}")
print(f"Лучший RMSE на кросс-валидации: {np.sqrt(-bayes_search.best_score_):.4f}")

# Предсказание на тестовой выборке
y_pred = bayes_search.predict(X_test)

# Расчет метрик
print("\nМетрики на тестовой выборке:")
print("----------------------------------------")
print(f"MAE (Средняя абсолютная ошибка): {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE (Средняя квадратичная ошибка): {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE (Корень из MSE): {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R² (Коэффициент детерминации): {r2_score(y_test, y_pred):.4f}")


Лучшие гиперпараметры:
max_depth: 14
max_features: 0.6447047672509045
min_samples_leaf: 1
min_samples_split: 10
n_estimators: 100
Лучший MSE на кросс-валидации: 1.4542
Лучший RMSE на кросс-валидации: 1.2059

Метрики на тестовой выборке:
----------------------------------------
MAE (Средняя абсолютная ошибка): 0.7428
MSE (Средняя квадратичная ошибка): 1.1256
RMSE (Корень из MSE): 1.0609
R² (Коэффициент детерминации): 0.6439


#### **Оценка метрик**

**MAE** (Средняя абсолютная ошибка): 0.7428- модель в среднем ошибается на ±0.74 мМ  значение ошибки снизилось

**MSE** (Средняя квадратичная ошибка): 1.0345 - метрика стала гораздо лучше

**RMSE** (Корень из MSE): 1.0171 - стала ближе по значению к **MAE**  выбросы стали играть меньшую роль

**R²** (Коэффициент детерминации): 0.6439 - модель обьясняет 64%, что уже можно рассматривать  как неплохую модель

### **Метод опорных векторов**

In [83]:
# Параметры для оптимизации
params = {
    'C': Real(0.1, 100, prior='log-uniform'),  # Параметр регуляризации
    'epsilon': Real(0.01, 1.0),               # Параметр допуска
    'gamma': Real(0.001, 1.0, prior='log-uniform')  # Параметр ядра 
}

# Модель SVR
SVR_model = SVR(kernel='linear')

# Байесовская оптимизация
bayes_search_SVR = BayesSearchCV(
    SVR_model,
    params,
    n_iter=32,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1  # Используем все ядра процессора
)

# Поиск лучших параметров
bayes_search_SVR.fit(X_train, y_train)

# Лучшие параметры
print("\nЛУЧШИЕ ГИПЕРПАРАМЕТРЫ:")
for key, val in bayes_search_SVR.best_params_.items():
    print(f'{key:<20}: {val}')
print(f"\nЛучший MSE на кросс-валидации:: {-bayes_search_SVR.best_score_:.4f}")
print(f"Лучший RMSE на кросс-валидации:: {np.sqrt(-bayes_search_SVR.best_score_):.4f}")

# Предсказания и оценка
y_pred = bayes_search_SVR.predict(X_test)

# Расчет метрик
print("\nМетрики на тестовой выборке:")
print("----------------------------------------")
print(f"MAE (Средняя абсолютная ошибка): {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE (Средняя квадратичная ошибка): {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE (Корень из MSE): {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R² (Коэффициент детерминации): {r2_score(y_test, y_pred):.4f}")


ЛУЧШИЕ ГИПЕРПАРАМЕТРЫ:
C                   : 0.147495182823931
epsilon             : 0.7568805381939806
gamma               : 0.0022333966578984483

Лучший MSE на кросс-валидации:: 2.1403
Лучший RMSE на кросс-валидации:: 1.4630

Метрики на тестовой выборке:
----------------------------------------
MAE (Средняя абсолютная ошибка): 1.1049
MSE (Средняя квадратичная ошибка): 2.6191
RMSE (Корень из MSE): 1.6184
R² (Коэффициент детерминации): 0.1714


#### **Оценка метрик**

**MAE** (Средняя абсолютная ошибка): 1.1049

**MSE** (Средняя квадратичная ошибка):  2.6191

**RMSE** (Корень из MSE): 1.6184

**R²** (Коэффициент детерминации): 0.1714

Модель показала себя примерно на равных с **LinearRegression**, и гораздо хуже чем  **RandomForest**

### **GradientBoosting**

In [84]:
# Параметры для оптимизации
param_space = {
    'n_estimators': Integer(800, 1200),
    'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
    'max_depth': Integer(3, 20),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Real(0.1, 1.0)
}

# Модель Gradient Boosting
model_GB = GradientBoostingRegressor(random_state=42)

# Байесовская оптимизация
bayes_search_GB = BayesSearchCV(
    model_GB,
    param_space,
    n_iter=32,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1  # Используем все ядра процессора
)

# Поиск лучших параметров
bayes_search_GB.fit(X_train, y_train)

# Лучшие параметры
print("\nЛУЧШИЕ ГИПЕРПАРАМЕТРЫ:")
for key, val in bayes_search_GB.best_params_.items():
    print(f'{key:<20}: {val}')
print(f"\nЛучший MSE на кросс-валидации:: {-bayes_search_GB.best_score_:.4f}")
print(f"Лучший RMSE на кросс-валидации:: {np.sqrt(-bayes_search_GB.best_score_):.4f}")

# Предсказания и оценка
y_pred = bayes_search_GB.predict(X_test)

# Расчет метрик
print("\nМетрики на тестовой выборке:")
print("----------------------------------------")
print(f"MAE (Средняя абсолютная ошибка): {mean_absolute_error(y_test, y_pred):.4f}")
print(f"MSE (Средняя квадратичная ошибка): {mean_squared_error(y_test, y_pred):.4f}")
print(f"RMSE (Корень из MSE): {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R² (Коэффициент детерминации): {r2_score(y_test, y_pred):.4f}")


ЛУЧШИЕ ГИПЕРПАРАМЕТРЫ:
learning_rate       : 0.01323928624909563
max_depth           : 3
max_features        : 0.3456798717333819
min_samples_leaf    : 1
min_samples_split   : 2
n_estimators        : 800

Лучший MSE на кросс-валидации:: 1.4581
Лучший RMSE на кросс-валидации:: 1.2075

Метрики на тестовой выборке:
----------------------------------------
MAE (Средняя абсолютная ошибка): 0.7796
MSE (Средняя квадратичная ошибка): 1.1751
RMSE (Корень из MSE): 1.0840
R² (Коэффициент детерминации): 0.6282


#### **Оценка метрик**

**MAE** (Средняя абсолютная ошибка): 0.7796

**MSE** (Средняя квадратичная ошибка):  1.1751

**RMSE** (Корень из MSE): 1.0840

**R²** (Коэффициент детерминации): 0.6282

**Все метрики немного хуже чем у RandomForest**

#### Наилучшая модель **RandomForest**
#### Соберем полный пайплайн для нашей модели и обучим ее

In [ ]:
# Создаем пайплайн
mod_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Стандартизируем
    ('random_forest', RandomForestRegressor(
        n_estimators=100,
        max_depth=14,
        max_features=0.6447047672509045,
        min_samples_split=10,
        min_samples_leaf=1,
        random_state=42 
    ))
])

In [88]:
encode_df = preprocessing_pipeline.fit_transform(df_regr1)
Y = encode_df['SI']
X = encode_df.drop(columns=['SI'])

mod_pipeline.fit(X, Y)


Pipeline(steps=[('scaler', StandardScaler()),
                ('random_forest',
                 RandomForestRegressor(max_depth=18,
                                       max_features=0.361554586956777,
                                       min_samples_split=10, n_estimators=1000,
                                       random_state=42))])

#### Модель  с лучшими метриками обучена и готова к Production.